In [7]:
execfile('../src/utils.py')
execfile('../src/cellslib.py')

In [8]:
# # ssC Data
# path = '/notebooks/moldev-data/original/06-19-2015/6.19.15 All ssC data - Jonah Simon.txt'
# savepath = '/notebooks/moldev-data/analyzed/06-19-2015/ssC.csv'

# APB Data
path = '/notebooks/moldev-data/original/06-19-2015/6.19.15 All APB data - Jonah Simon.txt'
savepath = '/notebooks/moldev-data/analyzed/06-19-2015/APB.csv'

conditions = [('PML ssC w RNase A',24),
              ('PML APB w RNase A',25),
              ('PML ssC wo RNase A',26),
              ('PML IF only control no EtOH dehydrat',27),
              ('PML ssC control no FISH',28),
              ('IgG APB w LNA probe before denature',29),
              ('IgG APB w LNA probe after denature',30),
              ('IgG ssC w LNA probe no denature',31)]

cell_dict = {"U2OS": ['A01','A02','B01','B02'],
             "143B": ['A03','A04','B03','B04']}

date = '06-18-2015'

In [9]:
well_df = create_well_df(cell_dict)
condition_lookup = pd.concat([create_plate_df(well_df,a,b) for a,b in conditions])
condition_lookup['Date'] = date

In [10]:
condition_lookup.head()

,Well Name,Plate Name,Condition,Plate ID,Date
0,A03,24,PML ssC w RNase A 143B,a54a2d0a0f37,06-18-2015
1,A04,24,PML ssC w RNase A 143B,a54a2d0a0f37,06-18-2015
2,B03,24,PML ssC w RNase A 143B,a54a2d0a0f37,06-18-2015
3,B04,24,PML ssC w RNase A 143B,a54a2d0a0f37,06-18-2015
0,A01,24,PML ssC w RNase A U2OS,a54a2d0a0f37,06-18-2015


In [11]:
# String -> String
def rename_column(col):
    """ Rename column col to remove whitespace, backslashes, prefixes,
        and suffixes (esp. large parenthetic suffix). """
    if col.startswith('Cell:'):
        return col.split('(')[0].lstrip("Cell:").rstrip('/').strip(' ')
    else:
        return col.split('(')[0].rstrip('/').strip(' ')

cell_config = dict(
    path = path,
    plate_delimiter = "ATF",
    delimiter = '\t',
    skiprows = 4,
    dropcols = ['Cell ID',
                'Site ID',
                'MEASUREMENT SET ID',
                '.*ObjectID.*',
                'Laser focus score',
                '\.[0-9]*\Z'],
    normcols = [['Normalized APB spots (by FITC & TxRed)',
                  ['# of APBs'],
                  ['# of FITC spots', '# of TxRed spots']],
                ['Normalized APB spots (by TxRed)',
                  ['# of APBs'],
                  ['# of TxRed spots']],
                ['Normalized APB spots (by FITC)',
                  ['# of APBs'],
                  ['# of FITC spots']],
                
                ['Normalized Coloc avg area (by FITC & TxRed)',
                  ['Area_Coloc_Avg'],
                  ['Area_FITC','Area_TxRed']],
                ['Normalized Coloc avg area (by FITC)',
                  ['Area_Coloc_Avg'],
                  ['Area_FITC']],
                ['Normalized Coloc avg area (by TxRed) ',
                  ['Area_Coloc_Avg'],
                  ['Area_TxRed']],                
                
                ['Normalized Coloc spots (by FITC & TxRed)',
                  ['# Coloc Spots'],
                  ['# of FITC spots', '# of TxRed spots']],
                ['Normalized Coloc spots (by FITC)',
                  ['# Coloc Spots'],
                  ['# of FITC spots']],
                ['Normalized Coloc spots (by TxRed)',
                  ['# Coloc Spots'],
                  ['# of TxRed spots']],
                
                ['Normalized Coloc total area (by FITC & TxRed)',
                  ['Total_Area_Coloc'],
                  ['Total_Area_FITC','Total_Area_TxRed']],
                ['Normalized Coloc total area (by FITC)',
                  ['Total_Area_Coloc'],
                  ['Total_Area_FITC']],
                ['Normalized Coloc total area (by TxRed) ',
                  ['Total_Area_Coloc'],
                  ['Total_Area_TxRed']],
                
                ['Normalized Coloc total area (by FITC-TxRed colocs) ',
                  ['Total_Area_Coloc'],
                  ['FITC-TxRed_Area_Total']]],
    colrename = rename_column,
    check = identity
    )

cell_data = get_cell_data(cell_config)

In [12]:
data = pd.merge(cell_data.rename(columns={"Plate ID":"Plate Name"}),
                condition_lookup,
                on = ['Plate Name','Well Name'])

# Write to file
data.to_csv(savepath,index=False)